In [1]:
import requests

import pandas as pd
import numpy as np
import copy
import json

from ast import literal_eval

import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel
from transformers import OwlViTProcessor, OwlViTForObjectDetection
from transformers import pipeline
from transformers import GPT2TokenizerFast
from PIL import Image

from tqdm import tqdm
tqdm.pandas()

import pickle

In [2]:
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict

import sklearn.datasets as datasets
import sklearn.manifold as manifold

In [3]:
import openai
import os
import sys
from dotenv import load_dotenv

load_dotenv()    
openai.api_key = os.getenv('OPENAI_API_KEY')

In [4]:
cur_os = sys.platform

In [5]:
# mps_device = torch.device('mps')

In [6]:
df = pd.read_csv('../data/recipes_after.csv')
df.shape

(4298, 7)

In [7]:
df.head(2)

,요리,종류,난이도,소요시간,링크,재료,재료수
0,꼬시래기 물냉면,메인요리,1,20,https://wtable.co.kr/recipes/GdRMuEBF9nXVJAtqw...,꼬시래기 통깨 고춧가루 용냉면육수 오이 달걀 쌈무 고추장 소금 식초 매실청 참기름,12
1,닭죽,메인요리,1,30,https://wtable.co.kr/recipes/QawUyVgt3e5wvCT9d...,통깨 찹쌀밥 다진당근 물 소금 닭가슴살 통마늘 참기름,8


In [8]:
df['feature'] = df['재료']
df.head(2)

,요리,종류,난이도,소요시간,링크,재료,재료수,feature
0,꼬시래기 물냉면,메인요리,1,20,https://wtable.co.kr/recipes/GdRMuEBF9nXVJAtqw...,꼬시래기 통깨 고춧가루 용냉면육수 오이 달걀 쌈무 고추장 소금 식초 매실청 참기름,12,꼬시래기 통깨 고춧가루 용냉면육수 오이 달걀 쌈무 고추장 소금 식초 매실청 참기름
1,닭죽,메인요리,1,30,https://wtable.co.kr/recipes/QawUyVgt3e5wvCT9d...,통깨 찹쌀밥 다진당근 물 소금 닭가슴살 통마늘 참기름,8,통깨 찹쌀밥 다진당근 물 소금 닭가슴살 통마늘 참기름


In [9]:
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [10]:
df['embeddings'] = df['feature'].progress_apply(lambda x : model.encode(x))
df.shape

100%|██████████| 4298/4298 [03:28<00:00, 20.63it/s]


(4298, 9)

In [11]:
df.head(2)

,요리,종류,난이도,소요시간,링크,재료,재료수,feature,embeddings
0,꼬시래기 물냉면,메인요리,1,20,https://wtable.co.kr/recipes/GdRMuEBF9nXVJAtqw...,꼬시래기 통깨 고춧가루 용냉면육수 오이 달걀 쌈무 고추장 소금 식초 매실청 참기름,12,꼬시래기 통깨 고춧가루 용냉면육수 오이 달걀 쌈무 고추장 소금 식초 매실청 참기름,"[-0.42411768, 0.5387487, 0.7363548, -0.1657364..."
1,닭죽,메인요리,1,30,https://wtable.co.kr/recipes/QawUyVgt3e5wvCT9d...,통깨 찹쌀밥 다진당근 물 소금 닭가슴살 통마늘 참기름,8,통깨 찹쌀밥 다진당근 물 소금 닭가슴살 통마늘 참기름,"[-0.05461666, 0.60962063, 0.9323036, -1.147650..."


In [12]:
def get_query_sim_top_k(query, model, df):
    query_encode = model.encode(query)
    cos_scores = util.pytorch_cos_sim(query_encode, df['embeddings'])[0]
    top_results = torch.topk(cos_scores, k=5)
    return top_results

In [13]:
query = '고기 쪽파'
top_result = get_query_sim_top_k(query, model, df)

/Users/choimingue/anaconda3/envs/torch/lib/python3.10/site-packages/sentence_transformers/util.py:39: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1678454852765/work/torch/csrc/utils/tensor_new.cpp:248.)
  b = torch.tensor(b)


In [14]:
top_result

torch.return_types.topk(
values=tensor([0.6566, 0.6546, 0.6475, 0.6429, 0.6380]),
indices=tensor([3337, 3339,  316, 1179, 2527]))

In [15]:
df.iloc[top_result[1].numpy(), :][['요리', '종류', '재료']]

,요리,종류,재료
3337,조염새우,중식,대파 간장 식용유 파기름 감자전분 마늘 백후춧가루 소금 설탕 중새우 사천고추 산초가...
3339,큐민향 양고기 볶음,중식,양고기 물전분 식용유 백후춧가루 설탕 소금 생추왕 쪽파 큐민가루 고수 산초가루 치킨파우더
316,완탕면,메인요리,면 새우 굴소스 백후추 삼겹살 소금 닭육수 설탕 쪽파 표고버섯 참기름 치킨파우더
1179,조염 옥수수,간식,식용유 백후추 감자전분 잣 옥수수 설탕 소금 쪽파 산초가루 치킨파우더
2527,가지 절임,채식,통깨 대파부분 채수 간장 식용유 비정제설탕 식초 쪽파 가지


In [16]:
def print_msg(msg):
    completion = openai.ChatCompletion.create(
                    model='gpt-3.5-turbo',
                    messages=msg
                    )
    return completion['choices'][0]['message']['content']

In [17]:
msg_prompt = {
    'recom' : {
                'system' : "당신은 사용자의 질문에 따라 레시피를 추천하는 유용한 도우미입니다.", 
                'user' : "사용자에게 레시피를 추천할 때, '👨🏻‍🍳 그럼요!'으로 시작하는 간단한 인사말 1문장을 작성하세요.", 
              },
    'desc' : {
                'system' : "You are a helpful assistant who kindly answers.", 
                'user' : "Please write a simple greeting starting with 'of course' to explain the recipes to the user.", 
              },
    'intent' : {
                'system' : "You are a helpful assistant who understands the intent of the user's question.",
                'user' : "Which category does the sentence below belong to: 'description', 'recommended', 'search'? Show only categories. \n context:"
                }
}

In [18]:
msg_prompt

{'recom': {'system': '당신은 사용자의 질문에 따라 레시피를 추천하는 유용한 도우미입니다.',
  'user': "사용자에게 레시피를 추천할 때, '👨🏻\u200d🍳 그럼요!'으로 시작하는 간단한 인사말 1문장을 작성하세요."},
 'desc': {'system': 'You are a helpful assistant who kindly answers.',
  'user': "Please write a simple greeting starting with 'of course' to explain the recipes to the user."},
 'intent': {'system': "You are a helpful assistant who understands the intent of the user's question.",
  'user': "Which category does the sentence below belong to: 'description', 'recommended', 'search'? Show only categories. \n context:"}}

In [19]:
user_msg_history = []

In [20]:
def get_chatgpt_msg(msg):
    completion = openai.ChatCompletion.create(
                    model='gpt-3.5-turbo',
                    messages=msg
                    )
    return completion['choices'][0]['message']['content']

In [21]:
def set_prompt(intent, query, msg_prompt_init, model):
    '''prompt 형태를 만들어주는 함수'''
    m = dict()
    # 검색 또는 추천이면
    if ('recom' in intent) or ('search' in intent):
        msg = msg_prompt_init['recom'] # 시스템 메세지를 가지고오고
    # 설명문이면
    elif 'desc' in intent:
        msg = msg_prompt_init['desc'] # 시스템 메세지를 가지고오고
    # intent 파악
    else:
        msg = msg_prompt_init['intent']
        msg['user'] += f' {query} \n A:'
    for k, v in msg.items():
        m['role'], m['content'] = k, v
    return [m]

In [22]:
def user_interact(query, model, msg_prompt_init):
    # 1. 사용자의 의도를 파악
    user_intent = set_prompt('intent', query, msg_prompt_init, None)
    user_intent = get_chatgpt_msg(user_intent).lower()
    print("user_intent : ", user_intent)
    
    # 2. 사용자의 쿼리에 따라 prompt 생성    
    intent_data = set_prompt(user_intent, query, msg_prompt_init, model)
    intent_data_msg = get_chatgpt_msg(intent_data).replace("\n", "").strip()
    print("intent_data_msg : ", intent_data_msg)
    
    # 3-1. 추천 또는 검색이면
    if ('recom' in user_intent) or ('search' in user_intent):
        recom_msg = str()
        # 기존에 메세지가 있으면 쿼리로 대체
        if (len(user_msg_history) > 0 ) and (user_msg_history[-1]['role'] == 'assistant'):
            query = user_msg_history[-1]['content']['feature']
        # 유사 아이템 가져오기
        #top_result = get_query_sim_top_k(query, model, movies_metadata, top_k=1 if 'recom' in user_intent else 3) # 추천 개수 설정하려면!
        top_result = get_query_sim_top_k(query, model, df)
        #print("top_result : ", top_result)
        # 검색이면, 자기 자신의 컨텐츠는 제외
        top_index = top_result[1].numpy() if 'recom' in user_intent else top_result[1].numpy()[1:]
        #print("top_index : ", top_index)
        # 장르, 제목, overview를 가져와서 출력
        r_set_d = df.iloc[top_index, :][['요리', '종류', '재료']]
        r_set_d = json.loads(r_set_d.to_json(orient="records"))
        for r in r_set_d:
            for _, v in r.items():
                recom_msg += f"{v} \n"
            recom_msg += "\n"
        user_msg_history.append({'role' : 'assistant', 'content' : f"{intent_data_msg} {str(recom_msg)}"})
        print(f"\nrecom data : {intent_data_msg} \n{str(recom_msg)}")
    # 3-2. 설명이면
    elif 'desc' in user_intent:
        # 이전 메세지에 따라서 설명을 가져와야 하기 때문에 이전 메세지 컨텐츠를 가져옴
        top_result = get_query_sim_top_k(user_msg_history[-1]['content'], model, df)
        # feature가 상세 설명이라고 가정하고 해당 컬럼의 값을 가져와 출력
        r_set_d = df.iloc[top_result[1].numpy(), :][['feature']]
        r_set_d = json.loads(r_set_d.to_json(orient="records"))[0]
        user_msg_history.append({'role' : 'assistant', 'content' : r_set_d})
        print(f"\n describe : {intent_data_msg} {r_set_d}")

In [23]:
query = "돼지고기와 양파가 들어간 음식을 추천해줘"
user_interact(query, model, copy.deepcopy(msg_prompt))

user_intent :  recommended
intent_data_msg :  👨🏻‍🍳 그럼요! 이번에 추천해드릴 레시피는 어떤 것이 좋을까요?

recom data : 👨🏻‍🍳 그럼요! 이번에 추천해드릴 레시피는 어떤 것이 좋을까요? 
육전 
메인요리 
간장 달걀 올리고당 쇠고기 영양부추 설탕 식초 양파 부침가루 사비 

육전 
안주 
간장 달걀 올리고당 쇠고기 영양부추 설탕 식초 양파 부침가루 사비 

육전 
초대요리 
간장 달걀 올리고당 쇠고기 영양부추 설탕 식초 양파 부침가루 사비 

쇠고기고추장스튜  
초대요리 
포도씨유 쇠고기 물 고추장 채소스톡큐브 감자 양파 

쇠고기고추장스튜  
간단요리 
포도씨유 쇠고기 물 고추장 채소스톡큐브 감자 양파 


